In [31]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to suppress all messages
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
label_encoder = LabelEncoder()
pd.options.mode.chained_assignment = None


In [46]:
raw_data_path = './data/prive_24h_data.csv'
flag_data_path = './data/ground_truth_11_21.csv'

data_df = pd.read_csv(raw_data_path)
flag_data_df = pd.read_csv(flag_data_path)
data_df['gender'] = label_encoder.fit_transform(data_df['gender'])
data_df['race'] = label_encoder.fit_transform(data_df['race'])
data_df['admission_type_x'] = label_encoder.fit_transform(data_df['admission_type_x'])
data_df['first_careunit'] = label_encoder.fit_transform(data_df['first_careunit'])
data_df['admission_type_y'] = label_encoder.fit_transform(data_df['admission_type_y'])
data_df['insurance'] = label_encoder.fit_transform(data_df['insurance'])

In [47]:
def check_missing_values(df):
    if df.isna().any().any():
        return 1 
    else:
        return 0 

In [73]:
total_x = []
total_y = []
for index, row in flag_data_df.iterrows():
    id_df = data_df[data_df['stay_id'] == row['stay_id']]
    if not check_missing_values(id_df):
        label = id_df['label'].iloc[0]
        if label == 1:
            label = -72
        else:
            if id_df['Rev_h'].iloc[0] != -1000:
                label = 48 - id_df['Rev_h'].iloc[0]
            elif id_df['dod_h'].iloc[0] != -1000 and id_df['dod_h'].iloc[0]>0 and id_df['dod_h'].iloc[0]<48:
                label = 96 - id_df['dod_h'].iloc[0]*2
            else:
                label = 96
        id_df = id_df.drop(columns='label')
        id_df = id_df.drop(columns='charttime')
        total_x.append(id_df.values)
        total_y.append(label)

total_x = np.array(total_x)
total_y = np.array(total_y).reshape(-1, 1)

print(total_x.shape)
print(total_y.shape)
        

(2811, 24, 27)
(2811, 1)


In [74]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def train_module(X_train, y_train, X_test, y_test, epoch, learning_rate, batch):

    # Define model and train
    def build_lstm_model(timesteps, num_features):
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(128, input_shape=(timesteps, num_features)),
            tf.keras.layers.Dense(1) 
        ])
        model.build(input_shape=(None, timesteps, num_features))
        return model

    model = build_lstm_model(timesteps=X_train.shape[1], num_features=X_train.shape[2])

    model.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), loss='mean_squared_error') 

    model.fit(X_train, y_train, epochs=epoch, batch_size=batch, validation_data=(X_test, y_test))
    return model



In [83]:
total_x = total_x.astype(np.float32)
total_y = total_y.astype(np.float32)
total_x[np.isinf(total_x)] = np.nan
total_x[np.abs(total_x) > 1e6] = np.nan
# Handle NaN values by replacing them with zeros (you can choose a different strategy)
total_x[np.isnan(total_x)] = 0.0
scaler = MinMaxScaler()
total_x_normalized = scaler.fit_transform(total_x.reshape(-1, total_x.shape[-1])).reshape(total_x.shape)

X_train, X_test, y_train, y_test = train_test_split(total_x_normalized, total_y, test_size=0.2, random_state=42)
model = train_module(X_train, y_train, X_test, y_test, epoch=300, learning_rate=0.001, batch=32)
evaluation_results = model.evaluate(X_train, y_train)
print("Evaluation results:", evaluation_results)

Epoch 1/200


2023-11-21 23:09:05.245721: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-21 23:09:05.355419: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 3/71 [>.............................] - ETA: 1s - loss: 5130.2007 

2023-11-21 23:09:05.456398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - ETA: 0s - loss: 4555.3398

2023-11-21 23:09:06.523825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-21 23:09:06.564757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - 2s 17ms/step - loss: 4555.3398 - val_loss: 4343.7388
Epoch 2/200
71/71 [==============================] - 1s 11ms/step - loss: 4348.4751 - val_loss: 4330.0430
Epoch 3/200
71/71 [==============================] - 1s 12ms/step - loss: 4332.3496 - val_loss: 4332.8901
Epoch 4/200
71/71 [==============================] - 1s 11ms/step - loss: 4328.2979 - val_loss: 4336.4727
Epoch 5/200
71/71 [==============================] - 1s 11ms/step - loss: 4326.8911 - val_loss: 4336.5903
Epoch 6/200
71/71 [==============================] - 1s 11ms/step - loss: 4327.7271 - val_loss: 4335.6699
Epoch 7/200
71/71 [==============================] - 1s 11ms/step - loss: 4326.2812 - val_loss: 4337.7422
Epoch 8/200
71/71 [==============================] - 1s 11ms/step - loss: 4325.2466 - val_loss: 4335.4932
Epoch 9/200
71/71 [==============================] - 1s 11ms/step - loss: 4320.5322 - val_loss: 4315.4131
Epoch 10/200
71/71 [==============================] - 1s 1

In [85]:
y_pred_proba = model.predict(X_test)
y_pred_proba = np.where(y_pred_proba > 0, 1, 0)
y_label = np.where(y_test > 0, 1, 0)
#print(y_pred_proba)
# Calculate AUROC
auroc = roc_auc_score(y_label, y_pred_proba)
print("AUROC:", auroc)

18/18 [==============================] - 0s 5ms/step
AUROC: 0.844043201696263
